In [1]:
import dgl
from scipy import spatial
import numpy as np
import pandas as pd

df = pd.read_excel('./dataset/Haircare/df.xlsx')
merged_df = pd.read_excel('./dataset/Haircare/merged_df.xlsx')
attributes = pd.read_excel('./dataset/Haircare/attributes.xlsx')
g = dgl.load_graphs('./dataset/Haircare/haircare_graph.dgl')[0][0]

saved_npz = np.load('./multisage/' + 'h_items.npz')
h_item = saved_npz['movie_vectors']
tree = spatial.KDTree(h_item.tolist())

In [2]:
entities = df['ID'].astype('category')
m_entities = entities.cat.reorder_categories(df['ID'].values)

index_id_to_ID = {}
ID_to_index_id = {}
for idx, ID in enumerate(m_entities):
    index_id_to_ID[idx] = ID
    ID_to_index_id[ID] = idx
    
h_query = h_item[ID_to_index_id[25320566538]]
index_ids = tree.query(h_query, 10)[1]
IDs = [index_id_to_ID[idx] for idx in index_ids]

for mid in IDs:
    print(df[df['ID']==mid]['상품명'].values)

['힘없는 모발용 트리트먼트 300ml']
['프리미엄 허브 샴푸 1000ml']
['퓨어 스칼프 샴푸 500ml']
['피에스바이오 케어팩 500ml']
['프리미엄 울트라 하드 미스트 250ml']
['프로 체인지 다크닝 샴푸 300g(신형)']
['초강력 헤어 스프레이 300ml']
['헤어 토닉 60ml']
['컬러팝 헤어 트리트먼트 50g']
['홍진단 편백 쿨링 샴푸 820ml']


In [3]:
import torch
import pickle
import sys

sys.path.append('C:/Users/kimhw/Desktop/데이터 청년 캠퍼스/Github/ESG-Recommender-System-by-MultiSAGE/multisage')
from model import MultiSAGEModel
from sampler import ItemToItemBatchSampler, NeighborSampler, PinSAGECollator

load_dict = torch.load('./multisage/MultiSAGE_weights.pth')

model = MultiSAGEModel(g, '상품', '특징', 512, 2, 3)
model.load_state_dict(load_dict)

batch_sampler = ItemToItemBatchSampler(g, '특징', '상품', 512)
neighbor_sampler = NeighborSampler(
    g, '특징', '상품', 2, 0.5, 10, 5, 2)
collator = PinSAGECollator(neighbor_sampler, g, '상품', '특징')

In [5]:
attributes[attributes['특징'] == '트리트먼트']

,Unnamed: 0,attribute_id,특징
86,86,86,트리트먼트


In [6]:
index_id = ID_to_index_id[25320566538]
with torch.no_grad():
    blocks, context_blocks = collator.collate_point(index_id=index_id)
    context_batch = model.get_representation(blocks, context_blocks, context_id=86)
    
tree.query(context_batch.numpy()[0], 10)

index_ids = tree.query(context_batch.numpy()[0], 10)[1]
IDs = [index_id_to_ID[idx] for idx in index_ids]
for mid in IDs:
    print(df[df['ID']==mid]['상품명'].values)

7
tensor([7, 7, 7, 0, 7])
execute context query
['힘없는 모발용 트리트먼트 300ml']
['프리미엄 허브 샴푸 1000ml']
['퓨어 스칼프 샴푸 500ml']
['피에스바이오 케어팩 500ml']
['초강력 헤어 스프레이 300ml']
['프리미엄 울트라 하드 미스트 250ml']
['프로 체인지 다크닝 샴푸 300g(신형)']
['홍진단 편백 쿨링 샴푸 820ml']
['컬러팝 헤어 트리트먼트 50g']
['콜라겐 2종 세트']
